In [8]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#monthly03 原版
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

urlretrieve = "http://localhost/monthly_03.html"#每月排名
data = requests.get(urlretrieve)
data.encoding = 'utf-8'
#print(data.apparent_encoding)#查看網頁目前編碼
#print(data.text)#查看網頁

soup = BeautifulSoup(data.text,"lxml")#提取內容

####################取得所有標題####################OK
monthly_title = []
monthly_title_str = soup.select(".blog-item.ranking h4.title a")
#print(len(monthly_title_str))#共20名
for i in range(len(monthly_title_str)):
    monthly_title.append(monthly_title_str[i].string)
#print("-"*10)
#print(monthly_title)

####################取得各項餐廳名稱、評級####################
monthly_restaurant = []
monthly_rating = []
monthly_restaurant_str = soup.select(".blog-item .restaurant.a, .blog-item .restaurant a")
monthly_rating_str = soup.select(".restaurant.item .rating")
#print(monthly_restaurant_str[0].text)
#print(monthly_rating_str[0].text)
for i in range(len(monthly_restaurant_str)//2):
    restaurant = monthly_restaurant_str[i*2].text if(i!=11)else monthly_title[11].string.split(" ")[1].split("-")[0]
    rating = "無紀錄" if(i==11)else "無紀錄" if(monthly_rating_str[i*2].text=="0")else monthly_rating_str[i*2].text
    monthly_restaurant.append(restaurant)
    monthly_rating.append(rating)#這有40格，取一半
#print("-"*10)
#print(monthly_restaurant)
#print(monthly_rating)

####################取得各項美食所在區域、產生日期####################OK
monthly_district = []
monthly_date = []
monthly_date_str = soup.select("h5, .h5")
#print(monthly_date_str)
for i in range(len(monthly_date_str)):
    date_DD_str = monthly_date_str[i].text.split(" ")
    #print(date_DD_str)
    district = date_DD_str[3].replace("(","").replace(")","") if(i!=13)else date_DD_str[4].replace("(","").replace(")","")
    date = date_DD_str[1].replace("。","") if(i!=13)else date_DD_str[2].replace("。","")
    monthly_district.append(district)
    monthly_date.append(date)
#print("-"*10)
#print(monthly_district)
#print("-"*10)
#print(monthly_date)

####################取得各項觀看、收藏、推薦次數####################
monthly_watch = []
monthly_collection = []
monthly_recommend = []
monthly_WCR_str = soup.select(".blog-item .stat-box")
#print(monthly_WCR_str)
for i in range(len(monthly_WCR_str)):
    WCR = monthly_WCR_str[i].text.replace("\n","").split(" ")
    monthly_watch.append(WCR[2])
    monthly_collection.append(WCR[4])
    monthly_recommend.append(WCR[6])
#print("-"*10)
#print(monthly_watch)
#print(monthly_collection)
#print(monthly_recommend)

####################取得各項來過人數、食記篇數####################
monthly_visit = []
monthly_foodrecord = []
monthly_VF_str = soup.select(".restaurant.right.item > .ng-binding")
#print(monthly_VF_str)
#print(monthly_VF_str[0].text)
#print(monthly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1])
for i in range(len(monthly_VF_str)):
    visit = monthly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[0] if(i!=11)else "無紀錄"
    foodrecord = monthly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1] if(i!=11)else "無紀錄"
    monthly_visit.append(visit)
    monthly_foodrecord.append(foodrecord)
#print("-"*10)
#print(monthly_visit)
#print(monthly_foodrecord)

###################弄成DataFrame####################
#原本
#cols = "第N名"
#rows = "標籤名稱(共十個)"
#反轉
#cols = "標籤名稱(共十個)"
#rows = "第N名"

monthly_DF = pd.DataFrame(
    {'標題': monthly_title,
     '餐廳名稱': monthly_restaurant,
     '評級': monthly_rating,
     '美食所在區域': monthly_district,
     '產生日期': monthly_date,
     '觀看人數': monthly_watch,
     '收藏人數': monthly_collection,
     '推薦人數': monthly_recommend,
     '多少人來過': monthly_visit,
     '多少篇食記': monthly_foodrecord,
    })

rows = {0:'第一名',1:'第二名',2:'第三名',3:'第四名',4:'第五名',5:'第六名',6:'第七名',7:'第八名',8:'第九名',9:'第十名',10:'第十一名',11:'第十二名',12:'第十三名',13:'第十四名',14:'第十五名',15:'第十六名',16:'第十七名',17:'第十八名',18:'第十九名',19:'第二十名'} 

monthly_DF = monthly_DF.rename(index=rows)
print(monthly_DF)
print("-"*10)
monthly_DF_reverse = monthly_DF.T
print(monthly_DF_reverse)
monthly_DF.to_csv('monthly_03.csv', encoding='utf_8_sig')
monthly_DF_reverse.to_csv('monthly_03_reverse.csv', encoding='utf_8_sig')

############################################

#print(soup.prettify())#把排版後的 html 印出來
#response.css('.blog-item.ranking h4.title').extract()#scrapy方法


                                                     標題  \
第一名   公館美食推薦，把厭世情緒都留在這裡吧！用美食美酒療癒身心！厭世會社The Misanthro...   
第二名               埔里知名鹹油條在台北開店囉！油條也能夾入料，立馬變身台式潛艇堡．普里鹹油條   
第三名   WOO Taiwan 大直Att美食 泰國觀光局推薦為此生必去十大名店 網美聚餐 泰國宮廷風...   
第四名                         手工現包整顆蝦仁皮薄晶透，蒸餃與外省麵必吃!!亓家蒸餃   
第五名                          粱記雞肉飯．據說台北最好吃、怎麼吃都吃不膩的雞肉飯！   
第六名          王品旗下 一鴨三吃不花大錢 桌邊片鴨秀 台北東區美食-享鴨-烤鴨與中華料理 含菜單！   
第七名                      濟善老麵 艋舺大道巷弄美食推薦，一碗雞湯拉麵吃進萬華人情風華   
第八名                  盈 豚骨カレー咖哩飯專門｜台北大安區｜從熬煮豚骨高湯開始製作的真咖哩   
第九名                             艋舺螺獅粉＿網路上好評不斷＿品嚐後絕對不會失望   
第十名                      北大行小籠包 信義區平價小吃推薦，原盅雞湯、上海點心非常好吃   
第十一名                     東輝韓食館 馬鈴薯豬骨湯名店！食尚玩家推薦行天宮巷弄韓式料理   
第十二名          中山站美食 米滋崎-ㄉㄨㄞ到爆的楓糖布丁乳酪！童年回憶啊！　中山站甜點/台北伴手禮   
第十三名           南港無名牛排│低調隱藏版牛排，搬家後吃貨照樣追著跑！ 南港玉成街牛排、後山埤牛排   
第十四名                         台北大安-秦家餅店-傳統北方烙餅，三代傳承的好味道    
第十五名                海鮮粥專賣店居然藏這味？！鮮蚵魯肉飯加半熟蛋，超邪惡爆棚美食．好吃食堂   
第十六名           華嫂冰室台北店｜港星最愛茶餐廳強勢登台,台北東區試營運中(國父紀念館捷運站週邊) 

In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#monthly04 原版
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

urlretrieve = "http://localhost/monthly_04.html"#每月排名
data = requests.get(urlretrieve)
data.encoding = 'utf-8'
#print(data.apparent_encoding)#查看網頁目前編碼
#print(data.text)#查看網頁

soup = BeautifulSoup(data.text,"lxml")#提取內容

####################取得所有標題####################OK
monthly_title = []
monthly_title_str = soup.select(".blog-item.ranking h4.title a")
#print(len(monthly_title_str))#共20名
for i in range(len(monthly_title_str)):
    monthly_title.append(monthly_title_str[i].string)
#print("-"*10)
#print(monthly_title)

####################取得各項餐廳名稱、評級####################
monthly_restaurant = []
monthly_rating = []
monthly_restaurant_str = soup.select(".blog-item .restaurant.a, .blog-item .restaurant a")
monthly_rating_str = soup.select(".restaurant.item .rating")
#print(monthly_restaurant_str[0].text)
#print(monthly_rating_str[0].text)
for i in range(len(monthly_restaurant_str)//2):
    restaurant = monthly_restaurant_str[i*2].text
    rating = monthly_rating_str[i*2].text if(monthly_rating_str[i*2].text!="0")else "無紀錄"
    monthly_restaurant.append(restaurant)
    monthly_rating.append(rating)#這有40格，取一半
#print("-"*10)
#print(monthly_restaurant)
#print(monthly_rating)

####################取得各項美食所在區域、產生日期####################OK
monthly_district = []
monthly_date = []
monthly_date_str = soup.select("h5, .h5")
#print(monthly_date_str)
for i in range(len(monthly_date_str)):
    date_DD_str = monthly_date_str[i].text.split(" ")
    #print(date_DD_str)
    district = date_DD_str[4].replace("(","").replace(")","") if(i==3)else date_DD_str[5].replace("(","").replace(")","") if(i==7)else date_DD_str[4].replace("(","").replace(")","") if(i==12)else date_DD_str[4].replace("(","").replace(")","") if(i==18)else date_DD_str[5].replace("(","").replace(")","") if(i==19)else date_DD_str[3].replace("(","").replace(")","")
    date = date_DD_str[2].replace("。","") if(i==3)else date_DD_str[3].replace("。","") if(i==7)else date_DD_str[2].replace("。","") if(i==12)else date_DD_str[2].replace("(","").replace(")","") if(i==18)else date_DD_str[3].replace("。","") if(i==19)else date_DD_str[1].replace("。","")
    monthly_district.append(district)
    monthly_date.append(date)
#print("-"*10)
#print(monthly_district)
#print("-"*10)
#print(monthly_date)

####################取得各項觀看、收藏、推薦次數####################
monthly_watch = []
monthly_collection = []
monthly_recommend = []
monthly_WCR_str = soup.select(".blog-item .stat-box")
#print(monthly_WCR_str)
for i in range(len(monthly_WCR_str)):
    WCR = monthly_WCR_str[i].text.replace("\n","").split(" ")
    monthly_watch.append(WCR[2])
    monthly_collection.append(WCR[4])
    monthly_recommend.append(WCR[6])
#print("-"*10)
#print(monthly_watch)
#print(monthly_collection)
#print(monthly_recommend)

####################取得各項來過人數、食記篇數####################
monthly_visit = []
monthly_foodrecord = []
monthly_VF_str = soup.select(".restaurant.right.item > .ng-binding")
#print(monthly_VF_str)
#print(monthly_VF_str[0].text)
#print(monthly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1])
for i in range(len(monthly_VF_str)):
    visit = monthly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[0]
    foodrecord = monthly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1]
    monthly_visit.append(visit)
    monthly_foodrecord.append(foodrecord)
#print("-"*10)
#print(monthly_visit)
#print(monthly_foodrecord)

###################弄成DataFrame####################
#原本
#cols = "第N名"
#rows = "標籤名稱(共十個)"
#反轉
#cols = "標籤名稱(共十個)"
#rows = "第N名"

monthly_DF = pd.DataFrame(
    {'標題': monthly_title,
     '餐廳名稱': monthly_restaurant,
     '評級': monthly_rating,
     '美食所在區域': monthly_district,
     '產生日期': monthly_date,
     '觀看人數': monthly_watch,
     '收藏人數': monthly_collection,
     '推薦人數': monthly_recommend,
     '多少人來過': monthly_visit,
     '多少篇食記': monthly_foodrecord,
    })

rows = {0:'第一名',1:'第二名',2:'第三名',3:'第四名',4:'第五名',5:'第六名',6:'第七名',7:'第八名',8:'第九名',9:'第十名',10:'第十一名',11:'第十二名',12:'第十三名',13:'第十四名',14:'第十五名',15:'第十六名',16:'第十七名',17:'第十八名',18:'第十九名',19:'第二十名'} 

monthly_DF = monthly_DF.rename(index=rows)
print(monthly_DF)
print("-"*10)
monthly_DF_reverse = monthly_DF.T
print(monthly_DF_reverse)
monthly_DF.to_csv('monthly_04.csv', encoding='utf_8_sig')
monthly_DF_reverse.to_csv('monthly_04_reverse.csv', encoding='utf_8_sig')

############################################

#print(soup.prettify())#把排版後的 html 印出來
#response.css('.blog-item.ranking h4.title').extract()#scrapy方法


                                                     標題  \
第一名                          台北美食面對面｜來自馬來西亞風味小吃台北1號店新開幕   
第二名                       Baba Kevin BBQ 近行天宮的最強美式烤肉餐廳！   
第三名                  行天宮美食，巷弄中的秒殺甜點，口味超多超平價的泡芙．諾緯手工鮮奶泡芙   
第四名   余文樂從小吃到大！香港來的超人氣美食～開幕每天都大排長龍不吃不行的「華嫂冰室」（菜單）｜Fa...   
第五名   烏石水產火鍋超市(烤)-金鋒滷肉飯、咖哩飯吃到飽！小蒙牛餐飲集團新推出！低消 $250 換湯...   
第六名                           台北 mumi cafe｜咖啡館裡我們一起烤吐司。   
第七名         流水蝦正夯，吃泰國蝦吃到厭世！10米水道流水泰國蝦無限吃到飽！泰蝦PA水道泰國蝦吃到飽   
第八名           死忠追隨ㄟ高CP值廚藝競賽冠軍泰式家鄉料理食堂 - Lisa 泰式美食｜民權西路站   
第九名                          普里鹹油條，來自埔里的特色早餐，僅販售早餐、消夜時段   
第十名   一個人也能吃的泰式料理，限量供應CNN評選世界50大美食第一名的巴東牛肉飯．Pretty T...   
第十一名         日本超人氣鰻魚飯台灣首發店，不用出國就能吃到道地京都鰻魚飯！江戶川鰻魚飯-台灣一號店   
第十二名                    捷運國父紀念館美食推薦|拿走雞蛋糕 巷子內的精品雞蛋糕不吃可惜   
第十三名                       台北京站 燒肉LIKE 一人燒肉店，單身也能輕鬆吃烤肉    
第十四名                     必吃隱藏版甜點，高CP無低消不限時韓式裱花蛋糕!!花甜居烘焙   
第十五名     中永和排隊早餐名店，上班遲到都要買！超豐滿爆料三明治！EVERYDAY CAFE-永和頂溪店   
第十六名                   南京復興站美食，兄弟飯店的壽司飯丸專賣店，超過20餘種的飯糰選擇 

In [15]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#monthly05 原版
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

urlretrieve = "http://localhost/monthly_05.html"#每月排名
data = requests.get(urlretrieve)
data.encoding = 'utf-8'
#print(data.apparent_encoding)#查看網頁目前編碼
#print(data.text)#查看網頁

soup = BeautifulSoup(data.text,"lxml")#提取內容

####################取得所有標題####################OK
monthly_title = []
monthly_title_str = soup.select(".blog-item.ranking h4.title a")
#print(len(monthly_title_str))#共20名
for i in range(len(monthly_title_str)):
    monthly_title.append(monthly_title_str[i].string)
#print("-"*10)
#print(monthly_title)

####################取得各項餐廳名稱、評級####################
monthly_restaurant = []
monthly_rating = []
monthly_restaurant_str = soup.select(".blog-item .restaurant.a, .blog-item .restaurant a")
monthly_rating_str = soup.select(".restaurant.item .rating") #7
#print(monthly_restaurant_str[0].text)
#print(monthly_rating_str[0].text)
for i in range(len(monthly_restaurant_str)//2):
    restaurant = monthly_restaurant_str[i*2].text# if(i!=11)else "無紀錄"
    rating = "無紀錄" if(monthly_rating_str[i*2].text=="0")else monthly_rating_str[i*2].text# if(i!=11)else "無紀錄"
    monthly_restaurant.append(restaurant)
    monthly_rating.append(rating)#這有40格，取一半
#print("-"*10)
#print(monthly_restaurant)
#print(monthly_rating)

####################取得各項美食所在區域、產生日期####################OK
monthly_district = []
monthly_date = []
monthly_date_str = soup.select("h5, .h5")
#print(monthly_date_str)
for i in range(len(monthly_date_str)): #2,14
    date_DD_str = monthly_date_str[i].text.split(" ")
    #print(date_DD_str)
    district = date_DD_str[4].replace("(","").replace(")","") if(i==2)else date_DD_str[4].replace("(","").replace(")","") if(i==14)else date_DD_str[3].replace("(","").replace(")","")
    date = date_DD_str[2].replace("。","") if(i==2)else date_DD_str[2].replace("。","") if(i==14)else date_DD_str[1].replace("。","")
    monthly_district.append(district)
    monthly_date.append(date)
#print("-"*10)
#print(monthly_district)
#print("-"*10)
#print(monthly_date)

####################取得各項觀看、收藏、推薦次數####################
monthly_watch = []
monthly_collection = []
monthly_recommend = []
monthly_WCR_str = soup.select(".blog-item .stat-box")
#print(monthly_WCR_str)
for i in range(len(monthly_WCR_str)):
    WCR = monthly_WCR_str[i].text.replace("\n","").split(" ")
    monthly_watch.append(WCR[2])
    monthly_collection.append(WCR[4])
    monthly_recommend.append(WCR[6])
#print("-"*10)
#print(monthly_watch)
#print(monthly_collection)
#print(monthly_recommend)

####################取得各項來過人數、食記篇數####################
monthly_visit = []
monthly_foodrecord = []
monthly_VF_str = soup.select(".restaurant.right.item > .ng-binding")
#print(monthly_VF_str)
#print(monthly_VF_str[0].text)
#print(monthly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1])
for i in range(len(monthly_VF_str)):
    visit = monthly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[0]# if(i!=11)else "無紀錄"
    foodrecord = monthly_VF_str[i].text.split("\n")[3].replace("\t","").replace("\r","").split(" ")[1]# if(i!=11)else "無紀錄"
    monthly_visit.append(visit)
    monthly_foodrecord.append(foodrecord)
#print("-"*10)
#print(monthly_visit)
#print(monthly_foodrecord)

###################弄成DataFrame####################
#原本
#cols = "第N名"
#rows = "標籤名稱(共十個)"
#反轉
#cols = "標籤名稱(共十個)"
#rows = "第N名"

monthly_DF = pd.DataFrame(
    {'標題': monthly_title,
     '餐廳名稱': monthly_restaurant,
     '評級': monthly_rating,
     '美食所在區域': monthly_district,
     '產生日期': monthly_date,
     '觀看人數': monthly_watch,
     '收藏人數': monthly_collection,
     '推薦人數': monthly_recommend,
     '多少人來過': monthly_visit,
     '多少篇食記': monthly_foodrecord,
    })

rows = {0:'第一名',1:'第二名',2:'第三名',3:'第四名',4:'第五名',5:'第六名',6:'第七名',7:'第八名',8:'第九名',9:'第十名',10:'第十一名',11:'第十二名',12:'第十三名',13:'第十四名',14:'第十五名',15:'第十六名',16:'第十七名',17:'第十八名',18:'第十九名',19:'第二十名'} 

monthly_DF = monthly_DF.rename(index=rows)
print(monthly_DF)
print("-"*10)
monthly_DF_reverse = monthly_DF.T
print(monthly_DF_reverse)
monthly_DF.to_csv('monthly_05.csv', encoding='utf_8_sig')
monthly_DF_reverse.to_csv('monthly_05_reverse.csv', encoding='utf_8_sig')

############################################

#print(soup.prettify())#把排版後的 html 印出來
#response.css('.blog-item.ranking h4.title').extract()#scrapy方法


                                                     標題  \
第一名   大阪爆紅的生吐司在台灣也吃得到囉！超濕潤的口感，吐司邊毫無存在感．Share le Pain...   
第二名     IG火紅冰品，來自日本的膠原蛋白牛奶冰，吸睛又美味．大とろ牛乳Ootoromilk (台北店)   
第三名              終於吃到必吃九宮格！搬到市區超方便！人氣依舊滿滿超好拍「吾獨食驗室」（菜單）   
第四名      台北中山:美天餐室 90年代舊香港茶餐廳，聽劉德華吃西多士、金山醬烤雞腿，再喝杯鴛鴦絲襪奶茶   
第五名                   王品集團最新品牌！懷舊料理說歷史，經典台菜大集合！丰禾日麗台式小館   
第六名                         小巷內的懷舊日本味，老爺爺的溫馨大阪燒！紅葉日式愛好燒   
第七名   浮誇又霸氣的料理/超平價無菜單竟然有鮮甜海膽 「東街日本料理」南京復興站/高cp值無菜單料理...   
第八名   IKEA新店店｜玻璃屋景觀餐廳、IKEA CAFÉ、挑高設計商場、與捷運相連，新店最美的IK...   
第九名       台北信義市府美食:HI5 Cafe好吃夏威夷蝦飯、家常料理，不限時咖啡館、推薦聊天聚餐開會   
第十名      好食多涮涮屋南西店新開幕，超大肉盤始祖、開幕優惠活龍蝦一隻僅 $300超划算😍 (文末菜單)   
第十一名                    雞窩餐廳-人少也能吃驥園砂鍋土雞湯,台北信義安和站中式聚餐推薦   
第十二名               西門麵店．捷運西門町站｜西門町網紅麵店．蛋包滷肉飯大推．極脆涼麵檸檬香！   
第十三名              BFF Gossip Brunch-台北公館網美最愛餐廳，姊妹聚會最佳選擇   
第十四名                      川畝園麵食館 平價好吃選擇多！推薦炒飯、牛肉捲餅、小籠湯包   
第十五名                             胖肚肚平價燒烤吃到飽，肋眼、天使紅蝦是哪招？   
第十六名          爆漿芋泥肉鬆起酥麵包，超ㄉㄨㄞ的芋頭水晶蛋糕，厚實的芋泥餡難以招架．土司吐司烘焙坊 